<a href="https://colab.research.google.com/github/Aapng-cmd/fish/blob/master/voice_synth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import librosa.display
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [87]:
from google.colab import files
files.upload()

!mv sample*.wav audio
NIGGA = 3

Saving sample0.wav to sample0.wav
Saving sample1.wav to sample1.wav
Saving sample2.wav to sample2.wav


In [88]:
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download('punkt')  # download the Punkt tokenizer models

def transcribe_audio(path):
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        try:
            text = r.recognize_google(audio_listened, language="ru-RU")
        except sr.UnknownValueError as e:
            print("Error:", str(e))
            text = ""
        return text

def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        text = transcribe_audio(chunk_filename)
        whole_text += text + " "
    sentences = sent_tokenize(whole_text)
    return ". ".join(sentences)

r = sr.Recognizer()
for i in range(NIGGA):
    path = f"audio/sample{i}.wav"
    text = get_large_audio_transcription_on_silence(path)
    with open(f"data/sample{i}.txt", "w") as f:
        f.write(text)

    print(text)

получив у трактирщика распоряжение по разборе запасов в погребе вы направились выполнять эту тяжёлую физическую работу
сам deploy был необычным потому что мы выкатываем большое изменение для обработки заявок основного договора
Мороз и день чудесный Ещё ты дремлешь друг прелестный


In [89]:
class MyDataset(Dataset):
    def __init__(self, audio_files, text_files):
        self.audio_files = audio_files
        self.text_files = text_files

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        text_file = self.text_files[idx]

        # Load audio file with error handling
        try:
            audio, sr = librosa.load(audio_file)
        except Exception as e:
            print(f"Error loading audio file: {audio_file}, {e}")
            audio = torch.zeros(1, 80)  # Replace with a placeholder if loading fails
            sr = 22050  # Default sample rate

        # Extract MFCC features from audio data
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=80)
        mfccs = mfccs.T  # Transpose to (time_steps, features)

        # Load text transcription
        with open(text_file, 'r') as f:
            text = f.read()

        # Tokenize text
        tokens = word_tokenize(text)

        # Convert tokens to numerical IDs (you'll need a vocabulary for this)
        vocab = {'<pad>': 0, '<unk>': 1}  # example vocabulary
        token_ids = [vocab.get(token, 1) for token in tokens]

        # Preprocess audio and text
        audio_tensor = torch.tensor(mfccs)  # (time_steps, features)
        text_tensor = torch.tensor(token_ids)

        return audio_tensor, text_tensor

In [90]:
import torch
import torch.nn as nn

class SimpleLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleLSTMModel, self).__init__()

        self.hidden_dim = hidden_dim

        self.i2h = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.i2o = nn.Linear(input_dim + hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_dim)

input_dim = 80  # Assuming 1-dimensional audio input
hidden_dim = 128  # Hidden layer size
output_dim = 1  # Assuming 1-dimensional audio output


In [96]:
dataset = MyDataset([f'audio/sample{i}.wav' for i in range(NIGGA)], [f'data/sample{i}.txt' for i in range(NIGGA)])
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [97]:
class FastSpeech(nn.Module):
    def __init__(self):
        super(FastSpeech, self).__init__()
        self.encoder = nn.LSTM(input_size=80, hidden_size=256, num_layers=2, batch_first=True)
        self.decoder = nn.Linear(256, 16)  # Output dimension matches the shape of the text tensor

    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.decoder(x[:, -1, :])  # Take the last hidden state and pass it through the decoder
        return x

In [98]:
model = FastSpeech()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    for batch in data_loader:
        audio, text = batch
        audio = audio.to(device)
        text = text.to(device)

        # Forward pass
        output = model(audio)

        # Calculate the loss
        loss = criterion(output, text.float())

        # Backpropagate the loss
        loss.backward()

        # Update the model parameters
        optimizer.step()

        # Reset the gradients
        optimizer.zero_grad()

        # Print the loss for the current batch
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

RuntimeError: stack expects each tensor to be equal size, but got [241, 80] at entry 0 and [330, 80] at entry 1

In [53]:
# Save the model after training
torch.save(model.state_dict(), 'models/fastspeech.pt')

In [56]:
!ls models
files.download("models/fastspeech.pt")

fastspeech.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>